In [1]:
import numpy as np

# Chapter 2<br>Systems of Equations

## 2.6 Methods for symmetric positive-definite matrices

In [2]:
def conj_grad_method(A, b, initial_guess=0, iter_num=0):
    n = len(A)
    x = initial_guess * np.ones(n)
    r = b - A.dot(x)
    d = r
    
    if iter_num == 0:
        iter_num = n

    for _ in range(iter_num):
        alpha = r.dot(r) / d.dot(A).dot(d)
        x = x + alpha*d
        beta = np.dot(r - alpha*A.dot(d), r - alpha*A.dot(d)) / r.dot(r)
        r = r - alpha*A.dot(d)
        d = r + beta*d
        
        if r.max() == 0:
            break
    
    return x

In [3]:
def Preconditioned_CGM(A, b, precondition, initial_guess=0, iter_num=0):
    n = len(A)
    
    if precondition == "Jacobi":
        M = np.diag(A)
        M_inv = np.diag(1/M)
    
    elif precondition == "Gauss_Seidel":
        D = np.diag(A)
        M = np.tril(A)/D*np.triu(A)
        M_inv = np.linalg.inv(M)
    
    else:
        raise ValueError("Please give the right preconditioner!")
    
    x = initial_guess * np.ones(n)
    r = b - A.dot(x)
    d = r.dot(M_inv)
    z = d
    
    if iter_num == 0:
        iter_num = n
    
    for _ in range(iter_num):
        alpha = r.dot(z) / d.dot(A).dot(d)
        x = x + alpha*d
        beta = np.dot(r - alpha*A.dot(d), (r - alpha*A.dot(d)).dot(M_inv)) / r.dot(z)
        r = r - alpha*A.dot(d)
        z = r.dot(M_inv)
        d = z + beta*d
        
        if r.max() == 0:
            break    
    
    return x

In [4]:
def Hilbert(n):
    H = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            H[i, j] = 1 / (i + j + 1)
            
    return H

### Q. 1

In [5]:
# (a)
A = np.array([[1, 0], [0, 2]])
b = np.array([2, 4])

In [6]:
x = conj_grad_method(A, b)
print("x:", x)

x: [2. 2.]


In [7]:
# (b)
A = np.array([[1, 2], [2, 5]])
b = np.array([1, 1])

In [8]:
x = conj_grad_method(A, b)
print("x:", x)

x: [ 3. -1.]


### Q. 2

In [9]:
# (a)
A = np.array([[1, -1, 0], [-1, 2, 1], [0, 1, 2]])
b = np.array([0, 2, 3])

In [10]:
x = conj_grad_method(A, b)
print("x:", x)

x: [1. 1. 1.]


In [11]:
# (b)
A = np.array([[1, -1, 0], [-1, 2, 1], [0, 1, 5]])
b = np.array([3, -3, 4])

In [12]:
x = conj_grad_method(A, b)
print("x:", x)

x: [ 2. -1.  1.]


### Q. 3

In [13]:
# (a)
n = 4
H = Hilbert(n)
b = np.ones(n)

In [14]:
x = conj_grad_method(H, b)
print("x:", x)

x: [  -4.   60. -180.  140.]


In [15]:
# (b)
n = 8
H = Hilbert(n)
b = np.ones(n)

In [16]:
x = conj_grad_method(H, b)
print("x:", x)

x: [   -5.64866393   153.38773204  -873.60128993  1378.10751732
   335.71392672 -1242.26504186 -1156.48123908  1453.93314816]


### Q. 4

In [17]:
def A_ij(n):
    A = np.zeros((n, n))
    for i in range(n):
        A[i, n-1-i] = 0.5
        A[i, i] = 3
        if i > 0:
            A[i, i-1] = -1
            A[i-1, i] = -1
            
    return A

In [18]:
# (a)
n = 6
A = A_ij(n)
b = 1.5 * np.ones(n)
b[0], b[-1] = 2.5, 2.5
b[n//2], b[n//2 - 1] = 1, 1

In [19]:
x = conj_grad_method(A, b)
print("x:", x)

x: [1. 1. 1. 1. 1. 1.]


In [20]:
# (b)
n = 12
A = A_ij(n)
b = 1.5 * np.ones(n)
b[0], b[-1] = 2.5, 2.5
b[n//2], b[n//2 - 1] = 1, 1

In [21]:
x = conj_grad_method(A, b)
print("x:", x)

x: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### Q. 5

In [22]:
tol = 1e-5

In [23]:
for n in [100, 1000, 10000]:
    A = A_ij(n)
    b = 1.5 * np.ones(n)
    b[0], b[-1] = 2.5, 2.5
    b[n//2], b[n//2 - 1] = 1, 1
    
    true_x = np.ones(n)
    x = np.zeros(n)

    iter_n = 0
    while True:
        x = conj_grad_method(A, b, initial_guess=x, iter_num=1)
        iter_n += 1
        if abs(x - true_x).max() < tol:
            break
    
    residue = abs(A.dot(x) - b)
    backward_error = residue.max()
    print("Iteration: %d / Backward Error: %f" % (iter_n, backward_error))

Iteration: 21 / Backward Error: 0.000013
Iteration: 21 / Backward Error: 0.000011
Iteration: 21 / Backward Error: 0.000011


### Q. 6

In [34]:
def A_ij(n):
    A = np.zeros((n, n))
    for i in range(n - 2):
        A[i, i] = i+1
        A[i, i+1] = 0.5
        A[i+1, i] = 0.5
        A[i, i+2] = 0.5
        A[i+2, i] = 0.5
        
    A[-2, -2] = n - 1
    A[-1, -1] = n
    A[-2, -1] = 0.5
    A[-1, -2] = 0.5
    
    return A

n = 1000
A = A_ij(n)

In [35]:
# (a)

In [36]:
# (b)
x_e = np.ones(n)
b = A.dot(x_e)
tol = 1e-4

Conjugate Gradient Method without preconditioner

In [39]:
iter_n = 0
x = np.zeros(n)

while True:
    x = conj_grad_method(A, b, initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

Iter: 5716 / Backward error: 0.000027


Jacobi preconditioner

In [40]:
iter_n = 0
x = np.zeros(n)

while True:
    x = Preconditioned_CGM(A, b, "Jacobi", initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

Iter: 10 / Backward error: 0.000009


Gauss-Seidel preconditioner

In [41]:
iter_n = 0
x = np.zeros(n)

while True:
    x = Preconditioned_CGM(A, b, "Gauss_Seidel", initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

Iter: 10 / Backward error: 0.000009


### Q. 7

In [46]:
def A_ij(n):
    A = np.zeros((n, n))
    for i in range(n - 2):
        A[i, i] = i+1
        A[i, i+1] = 0.5
        A[i+1, i] = 0.5
        A[i, i+2] = 0.5
        A[i+2, i] = 0.5
        
    A[-2, -2] = n - 1
    A[-1, -1] = n
    A[-2, -1] = 0.5
    A[-1, -2] = 0.5
    
    for i in range(n//2):
        A[i, 2*i + 1] = 0.5
        A[2*i + 1, i] = 0.5
    
    return A

n = 1000
A = A_ij(n)

In [47]:
# (a)

In [48]:
# (b)
x_e = np.ones(n)
b = A.dot(x_e)
tol = 1e-4

Conjugate Gradient Method without preconditioner

In [49]:
iter_n = 0
x = np.zeros(n)

while True:
    x = conj_grad_method(A, b, initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

Iter: 5725 / Backward error: 0.000062


Jacobi preconditioner

In [50]:
iter_n = 0
x = np.zeros(n)

while True:
    x = Preconditioned_CGM(A, b, "Jacobi", initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

Iter: 10 / Backward error: 0.000008


Gauss-Seidel preconditioner

In [51]:
iter_n = 0
x = np.zeros(n)

while True:
    x = Preconditioned_CGM(A, b, "Gauss_Seidel", initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

Iter: 10 / Backward error: 0.000008


### Q. 8

In [63]:
def A_ij(n):
    A = np.zeros((n, n))
    
    for i in range(n):
        try:
            A[i, i] = 2
            A[i, i+2] = 0.5
            A[i+2, i] = 0.5
            A[i, i+4] = 0.5
            A[i+4, i] = 0.5
        except:
            pass
        
    for i in range(n-5):
        A[-1, i] = -0.1
        A[i, -1] = -0.1

    return A

n = 500
A = A_ij(n)

In [64]:
# (a)

In [65]:
# (b)
x_e = np.ones(n)
b = A.dot(x_e)
tol = 1e-4

Conjugate Gradient Method without preconditioner

In [66]:
iter_n = 0
x = np.zeros(n)

while True:
    x = conj_grad_method(A, b, initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

Iter: 94 / Backward error: 0.000014


Jacobi preconditioner

In [67]:
iter_n = 0
x = np.zeros(n)

while True:
    x = Preconditioned_CGM(A, b, "Jacobi", initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

Iter: 94 / Backward error: 0.000014


Gauss-Seidel preconditioner

In [68]:
iter_n = 0
x = np.zeros(n)

while True:
    x = Preconditioned_CGM(A, b, "Gauss_Seidel", initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

Iter: 94 / Backward error: 0.000014


### Q. 9

In [69]:
def A_ij(n):
    A = np.zeros((n, n))
    
    for i in range(n):
        try:
            A[i, i] = i**(1/3)
            A[i, i+2] = 0.5
            A[i+2, i] = 0.5
            A[i, i+4] = 0.5
            A[i+4, i] = 0.5
        except:
            pass
        
    for i in range(n-5):
        A[-1, i] = -0.1
        A[i, -1] = -0.1

    return A

n = 500
A = A_ij(n)

In [70]:
# (a)

In [75]:
# (b)
x_e = np.ones(n)
b = A.dot(x_e)
tol = 0.1

Conjugate Gradient Method without preconditioner

In [77]:
iter_n = 0
x = np.zeros(n)

while True:
    x = conj_grad_method(A, b, initial_guess=x, iter_num=1)
    iter_n += 1
    error = abs(x - x_e).max()
    print(error)
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

5.335627649925293
0.8828401079462498
0.8359495119362281
0.8041134902495616
0.792043736030862
0.7906137651885408
0.7991276695088363
0.8212279079634464
0.842838631354801
0.8815133176314545
0.9117566945138353
0.9621037278239738
0.9987593477628306
1.0583464821023334
1.1005885328923846
1.1686213006782864
1.216325299089274
1.2928405729589687
1.3462289473773421
1.4316930419165272
1.4911838586656652
1.5863206570288468
1.6524644248379228
1.7581834108396595
1.831637257360336
1.9490050659098184
2.030523912166372
2.1607563186460483
2.2511925761114187
2.3956571025959557
2.495965158750925
2.656189893387668
2.767433662028461
2.945120243348421
3.0684829702964045
3.2655227836651326
3.4023187439951923
3.620811922196006
3.772499877417375
4.0147769875852966
4.182975401541857
4.451621867602952
4.638125959464124
4.936009374436184
5.1428101409562945
5.473110697311512
5.702416077279281
6.068660401780053
6.322918790072935
6.729017520590593
7.01094387131553
7.461233361986726
7.773838152095552
8.273126742697857


179107539922.12128
190611079720.81693
198597183109.37674
211352484202.79205
220207597938.80704
234350871125.21616
244169556842.46954
259851834740.924
270738943827.0667
288127693718.4767
300199486997.8248
319480399167.84894
332865788422.8166
354244759103.8446
369086683692.92084
392792013780.51105
409248967053.35474
435533800076.69037
453781521886.56226
482926547267.4287
503159900654.89557
535476351122.632
557911403211.1968
593744378382.2966
618620707707.3403
658352859321.4095
685936114231.8562
729991732397.679
760576467850.9407
809426011938.6938
843338834982.3488
897503957546.2755
935107015076.322
995166132457.1274
1036860978497.7476
1103455447591.8442
1149687331394.429
1223528298550.2654
1274790919303.2812
1356666914482.4766
1413507693406.5793
1504293050706.0842
1567318976833.918
1667983171289.9392
1737867283356.2983
1849485283735.473
1926973857396.379
2050737605529.1711
2136658122660.4475
2273889261901.8125
2369159247079.3877
2521323235820.8784
2626960054345.4663
2795681815293.592
291

2.8189850551244475e+22
2.9370931217384274e+22
3.1257337989153033e+22
3.2566938318777906e+22
3.465861503565217e+22
3.6110719936292863e+22
3.843000311179358e+22
4.004011926308434e+22
4.2611775965463755e+22
4.439709741125155e+22
4.7248589745069385e+22
4.922818150448043e+22
5.238995987182577e+22
5.458496153002795e+22
5.8090789803053076e+22
6.052464125581106e+22
6.441195733301663e+22
6.711064909571166e+22
7.142096469227728e+22
7.441331544638151e+22
7.91926593878066e+22
8.251062372866414e+22
8.781003348154559e+22
9.148904315382507e+22
9.736510984271062e+22
1.0144445210750055e+23
1.0795992483792232e+23
1.124831620119607e+23
1.1970761795306935e+23
1.2472305260026644e+23
1.327336399827174e+23
1.3829483072563985e+23
1.4717709268902796e+23
1.533434261485717e+23
1.63192214236083e+23
1.7002953920694108e+23
1.8095002626221192e+23
1.8853135689634515e+23
2.0064016017901106e+23
2.0904645568624854e+23
2.2247288219966422e+23
2.3179391139166965e+23
2.466813387213556e+23
2.5701663863121675e+23
2.7352404603

2.6420184822427447e+31
2.75271211445227e+31
2.9295105528470015e+31
3.0522493473975593e+31
3.248286163371903e+31
3.3843808183851992e+31
3.6017494420353503e+31
3.7526532796606966e+31
3.993674938335997e+31
4.1609994244286515e+31
4.428247930561209e+31
4.613779883139388e+31
4.910109119369157e+31
5.115829789614698e+31
5.44440418471707e+31
5.672510414281175e+31
6.036838735342209e+31
6.28976641589005e+31
6.693739237587846e+31
6.974189323100889e+31
7.422120574218586e+31
7.733087923833749e+31
8.22976095466938e+31
8.574566314060799e+31
9.1252849766768e+31
9.507610439501688e+31
1.0118255726287813e+32
1.0542184054380682e+32
1.1219276899759904e+32
1.1689335121966065e+32
1.2440106037887836e+32
1.2961313793117661e+32
1.3793780081959225e+32
1.4371703222707028e+32
1.529475459212083e+32
1.5935564620867486e+32
1.6959058114835087e+32
1.7669598088041467e+32
1.8804463347879893e+32
1.9592320951344025e+32
2.0850676930721565e+32
2.172426550665374e+32
2.3119549886986824e+32
2.40881982780713e+32
2.563531096629844

3.1719205665410124e+40
3.37564350579511e+40
3.517074098810404e+40
3.7429652136580453e+40
3.899785621054289e+40
4.1502571484823623e+40
4.3241420177430355e+40
4.601868683063475e+40
4.794674888963726e+40
5.102622468563014e+40
5.316408942289613e+40
5.657865934442826e+40
5.89491565042603e+40
6.27352839238047e+40
6.536372747629903e+40
6.9561843539652e+40
7.247630200250818e+40
7.713123738328917e+40
8.036283355877231e+40
8.55242971944252e+40
8.910754052229987e+40
9.483065044390257e+40
9.880380551945553e+40
1.0514967744394196e+41
1.0955517263640919e+41
1.1659157260663865e+41
1.2147645314157494e+41
1.292785211836355e+41
1.3469495152758415e+41
1.4334600405309285e+41
1.4935182496540106e+41
1.5894424448746213e+41
1.6560359068786452e+41
1.76239812351749e+41
1.8362379739964313e+41
1.954174028631142e+41
2.0360487856218948e+41
2.1668180890675855e+41
2.2576020734447838e+41
2.402601079699807e+41
2.50326375183855e+41
2.6640408704815038e+41
2.775657182891967e+41
2.953929314175867e+41
3.077691190663102e+41


...

1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.29386791107010

1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.29386791107010

1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.29386791107010

1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.2938679110701066e+154
1.29386791107010

KeyboardInterrupt: 

Jacobi preconditioner

In [78]:
iter_n = 0
x = np.zeros(n)

while True:
    x = Preconditioned_CGM(A, b, "Jacobi", initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

C:\Users\snuist\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
C:\Users\snuist\Anaconda3\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


KeyboardInterrupt: 

Gauss-Seidel preconditioner

In [ ]:
iter_n = 0
x = np.zeros(n)

while True:
    x = Preconditioned_CGM(A, b, "Gauss_Seidel", initial_guess=x, iter_num=1)
    iter_n += 1
    if abs(x - x_e).max() < tol:
        break
        
backward_error = (A.dot(x) - b).max()
print("Iter: %d / Backward error: %f" % (iter_n, backward_error))

### Q. 10